In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from data.three_stocks import prices, composition
from configuration.momentum import three_stocks_config as c

In [2]:
c # pour les décisions au jour t, on regarde les returns entre les jours t-7 et t-2

{'t1': 7, 't2': 2, 'mp': 1, 'lsr': 0.1}

In [3]:
display(prices)
display(composition)

,stock_a,stock_b,stock_c
t0,100.25,75.6,55.8
t1,105.50,78.9,58.2
t2,98.75,76.3,57.1
t3,102.30,79.1,56.4
t5,99.80,80.2,59.3
t8,101.75,77.5,61.2
t9,104.20,82.1,60.7
t11,97.90,81.3,62.8
t12,103.40,83.2,63.5
t13,106.60,85.1,65.0


,stock_a,stock_b,stock_c
t0,True,False,True
t1,False,True,False
t2,True,True,True
t3,False,True,True
t4,True,False,True
t6,False,True,True
t7,True,True,True
t8,True,True,False
t9,True,False,True
t11,False,True,False


In [4]:
ret = prices / prices.shift() - 1
display(ret)
presence = composition.reindex_like(ret).fillna(False)
display(presence)

,stock_a,stock_b,stock_c
t0,NaN,NaN,NaN
t1,0.052369,0.043651,0.043011
t2,-0.063981,-0.032953,-0.018900
t3,0.035949,0.036697,-0.012259
t5,-0.024438,0.013906,0.051418
t8,0.019539,-0.033666,0.032040
t9,0.024079,0.059355,-0.008170
t11,-0.060461,-0.009744,0.034596
t12,0.056180,0.023370,0.011146
t13,0.030948,0.022837,0.023622


,stock_a,stock_b,stock_c
t0,True,False,True
t1,False,True,False
t2,True,True,True
t3,False,True,True
t5,False,False,False
t8,True,True,False
t9,True,False,True
t11,False,True,False
t12,False,False,False
t13,False,False,False


In [5]:
display(ret.rolling(c['t1'] - c['t2'], min_periods=c['mp']).mean())
display(ret.rolling(c['t1'] - c['t2'], min_periods=c['mp']).mean().shift(c['t2']))
historical_ret = ret.rolling(c['t1'] - c['t2'], min_periods=c['mp']).mean().shift(c['t2'])[presence]
display(historical_ret)

,stock_a,stock_b,stock_c
t0,NaN,NaN,NaN
t1,0.052369,0.043651,0.043011
t2,-0.005806,0.005349,0.012055
t3,0.008112,0.015798,0.003950
t5,-0.000025,0.015325,0.015817
t8,0.003888,0.005527,0.019062
t9,-0.001770,0.008668,0.008826
t11,-0.001066,0.013310,0.019525
t12,0.002980,0.010644,0.024206
t13,0.014057,0.012430,0.018647


,stock_a,stock_b,stock_c
t0,NaN,NaN,NaN
t1,NaN,NaN,NaN
t2,NaN,NaN,NaN
t3,0.052369,0.043651,0.043011
t5,-0.005806,0.005349,0.012055
t8,0.008112,0.015798,0.003950
t9,-0.000025,0.015325,0.015817
t11,0.003888,0.005527,0.019062
t12,-0.001770,0.008668,0.008826
t13,-0.001066,0.013310,0.019525


,stock_a,stock_b,stock_c
t0,NaN,NaN,NaN
t1,NaN,NaN,NaN
t2,NaN,NaN,NaN
t3,NaN,0.043651,0.043011
t5,NaN,NaN,NaN
t8,0.008112,0.015798,NaN
t9,-0.000025,NaN,0.015817
t11,NaN,0.005527,NaN
t12,NaN,NaN,NaN
t13,NaN,NaN,NaN


In [7]:
# Pourquoi 3 NaN pour commencer ?
# Parce que il y a 1 NaN dû aux returns (le jour 0 est NaN car on calcule les returns comme r_t = p_t / p_{t-1} - 1), et de plus, on doit se décaler de t2 = 2 jours

# Intuition: Pourquoi on shift de t2 jours ?
# Car on regarde les returns entre t-t1 et t-t2 pour décider à la date t.
# Si on se contentait de faire rolling(t1-t2), cela reviendrait au même que de regarder
# les returns entre t-(t1-t2) et t pour décider à la date t.
# Comme ce n'est pas ce qu'on veut, on décale de t2 jours.
# En revanche, pour le mean reversion, on peut considérer les returns entre t-t1 et t-t2 où t2=0, i.e. entre t-t1 et t